# Pre-trained ResNet

In [47]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

import os
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

from PIL import Image
from sklearn.model_selection import GroupShuffleSplit

In [5]:
import torch
import torch.nn as nn
from torchvision import models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

NUM_CLASSES = 7  # HAM10000 has 7 classes

# Load a pretrained ResNet
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Replace the last layer for our dataset classes
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, NUM_CLASSES)

if torch.cuda.device_count() > 1:
    print(f"ResNet Activated on {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(DEVICE)
print("ResNet Model Initialized!")


DEVICE: cuda
ResNet Activated on 2 GPUs!
ResNet Model Initialized!


# Dataset import

In [10]:
DATA_DIR = "/kaggle/input/skin-cancer-mnist-ham10000"

metadata_path = os.path.join(DATA_DIR, "HAM10000_metadata.csv")
img_dir_1 = os.path.join(DATA_DIR, "ham10000_images_part_1")
img_dir_2 = os.path.join(DATA_DIR, "ham10000_images_part_2")

if not os.path.exists(img_dir_1):
    img_dir_1 = os.path.join(DATA_DIR, "HAM10000_images_part_1")
if not os.path.exists(img_dir_2):
    img_dir_2 = os.path.join(DATA_DIR, "HAM10000_images_part_2")

assert os.path.exists(metadata_path), f"Metadata not found at: {metadata_path}"
assert os.path.exists(img_dir_1), f"Image folder not found: {img_dir_1}"
assert os.path.exists(img_dir_2), f"Image folder not found: {img_dir_2}"

print("Found metadata:", metadata_path)
print("Found images:", img_dir_1)
print("Found images:", img_dir_2)


Found metadata: /kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv
Found images: /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_1
Found images: /kaggle/input/skin-cancer-mnist-ham10000/ham10000_images_part_2


## Read metadata

In [16]:
df = pd.read_csv(metadata_path)

# Expected columns: image_id, lesion_id, dx (label)
required_cols = {"image_id", "lesion_id", "dx"}
missing = required_cols - set(df.columns)
assert not missing, f"Missing columns in metadata: {missing}"

## Build image_id -> filepath map

In [18]:
all_images = glob.glob(os.path.join(img_dir_1, "*.jpg")) + glob.glob(os.path.join(img_dir_2, "*.jpg"))
id2path = {os.path.splitext(os.path.basename(p))[0]: p for p in all_images}

# Attach image paths
df["path"] = df["image_id"].map(id2path)
df = df.dropna(subset=["path"]).reset_index(drop=True)

print("Total images found:", len(all_images))
print("Metadata rows with valid paths:", len(df))

Total images found: 10015
Metadata rows with valid paths: 10015


##  Label encoding

In [21]:
classes = sorted(df["dx"].unique().tolist())
class_to_idx = {c: i for i, c in enumerate(classes)}
idx_to_class = {i: c for c, i in class_to_idx.items()}

df["label"] = df["dx"].map(class_to_idx).astype(int)

print("Classes:", classes)
print("Class counts:\n", df["dx"].value_counts())

#Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Class counts:
 dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64


# Split data

In [26]:
# divide into 70%, 30%
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.30, random_state=42)
train_idx, temp_idx = next(gss1.split(df, groups=df["lesion_id"]))

train_df = df.iloc[train_idx].reset_index(drop=True)  # 70%
temp_df = df.iloc[temp_idx].reset_index(drop=True)   # 30%

# split that 30% of test to further 15, 15 for test and validation
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=42)
val_idx, test_idx = next(gss2.split(temp_df, groups=temp_df["lesion_id"]))

val_df = temp_df.iloc[val_idx].reset_index(drop=True)
test_df = temp_df.iloc[test_idx].reset_index(drop=True)

print("\nSplit sizes:")
print("Train:", len(train_df))
print("Val:  ", len(val_df))
print("Test: ", len(test_df))

# Quick safety check: no lesion_id overlap
train_lesions = set(train_df["lesion_id"])
val_lesions = set(val_df["lesion_id"])
test_lesions = set(test_df["lesion_id"])

assert train_lesions.isdisjoint(val_lesions)
assert train_lesions.isdisjoint(test_lesions)
assert val_lesions.isdisjoint(test_lesions)

print("Lesion-level split verified (no leakage).")


Split sizes:
Train: 7002
Val:   1519
Test:  1494
Lesion-level split verified (no leakage).


# Transforms (ResNet expects 224x224 + ImageNet normalization)

In [30]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

eval_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


# Dataset class

In [37]:
class HAM10000Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        y = int(row["label"])

        if self.transform:
            img = self.transform(img)

        return img, y

train_ds = HAM10000Dataset(train_df, transform=train_tfms)
val_ds   = HAM10000Dataset(val_df, transform=eval_tfms)
test_ds  = HAM10000Dataset(test_df, transform=eval_tfms)

# DataLoaders

In [39]:
BATCH_SIZE = 64 
NUM_WORKERS = 2

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

print("\n DataLoaders ready!")
print("Batches per epoch:", len(train_loader))


 DataLoaders ready!
Batches per epoch: 110


# Class weights

In [42]:
class_counts = train_df["label"].value_counts().sort_index().values  # count per class index
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)
class_weights = class_weights / class_weights.sum() * len(class_counts)  # normalize around num_classes
class_weights = class_weights.to(DEVICE)

print("Class counts:", class_counts)
print("Class weights:", class_weights.detach().cpu().numpy())

criterion = nn.CrossEntropyLoss(weight=class_weights)

Class counts: [ 207  381  741   89  770 4718   96]
Class weights: [1.057808   0.5747146  0.29550102 2.460295   0.28437176 0.04641082
 2.2808986 ]


# Optimizer + scheduler

In [43]:
LR = 3e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# Scheduler: reduce LR when val loss plateaus
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2, verbose=True
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# Train / Eval functions

In [48]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, desc="Train", leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


@torch.no_grad()
def evaluate(model, loader, criterion, device, desc="Val"):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, desc=desc, leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        logits = model(images)
        loss = criterion(logits, labels)

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# training loop

In [49]:
EPOCHS = 10  # start with 10; can increase later

best_val_acc = 0.0
best_path = "best_resnet_ham10000.pt"

history = {
    "train_loss": [], "train_acc": [],
    "val_loss": [], "val_acc": []
}

for epoch in range(1, EPOCHS + 1):
    print(f"\nEpoch {epoch}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, DEVICE)
    val_loss, val_acc = evaluate(model, val_loader, criterion, DEVICE, desc="Val")

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f}")

    scheduler.step(val_loss)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "model_state": model.state_dict() if not isinstance(model, nn.DataParallel) else model.module.state_dict(),
            "class_to_idx": class_to_idx,
            "idx_to_class": idx_to_class,
            "classes": classes
        }, best_path)
        print(f"Saved best model to {best_path} (val_acc={best_val_acc:.4f})")

print("\nTraining done. Best Val Acc:", best_val_acc)


Epoch 1/10


Train: loss=1.2313, acc=0.5453
Val:   loss=1.1801, acc=0.5951
Saved best model to best_resnet_ham10000.pt (val_acc=0.5951)

Epoch 2/10


Train: loss=0.9050, acc=0.6561
Val:   loss=1.2097, acc=0.5932

Epoch 3/10


Train: loss=0.7690, acc=0.6925
Val:   loss=1.0333, acc=0.6406
Saved best model to best_resnet_ham10000.pt (val_acc=0.6406)

Epoch 4/10


Train: loss=0.6615, acc=0.7142
Val:   loss=0.6986, acc=0.7505
Saved best model to best_resnet_ham10000.pt (val_acc=0.7505)

Epoch 5/10


...

Train: loss=0.6149, acc=0.7191
Val:   loss=1.2204, acc=0.6228

Epoch 7/10


Train: loss=0.5538, acc=0.7334
Val:   loss=0.8733, acc=0.6880

Epoch 8/10


Train: loss=0.3902, acc=0.7909
Val:   loss=0.9767, acc=0.7024

Epoch 9/10


Train: loss=0.3413, acc=0.8033
Val:   loss=0.8491, acc=0.7215

Epoch 10/10


Train: loss=0.3060, acc=0.8268
Val:   loss=0.6998, acc=0.7841
Saved best model to best_resnet_ham10000.pt (val_acc=0.7841)

Training done. Best Val Acc: 0.7840684660961159


# Testing

In [51]:
best_path = "best_resnet_ham10000.pt"
ckpt = torch.load(best_path, map_location=DEVICE)

# restore weights (handles DataParallel vs not)
if isinstance(model, nn.DataParallel):
    model.module.load_state_dict(ckpt["model_state"])
else:
    model.load_state_dict(ckpt["model_state"])

model = model.to(DEVICE)
model.eval()

# restore label maps (useful for reporting)
idx_to_class = ckpt["idx_to_class"]
classes = ckpt["classes"]

print("Loaded best model:", best_path)
print("Classes:", classes)

all_preds = []
all_labels = []

@torch.no_grad()
def predict_all(model, loader, device):
    preds_list, labels_list = [], []
    for images, labels in loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        logits = model(images)
        preds = torch.argmax(logits, dim=1)

        preds_list.append(preds.detach().cpu())
        labels_list.append(labels.detach().cpu())

    preds_list = torch.cat(preds_list).numpy()
    labels_list = torch.cat(labels_list).numpy()
    return preds_list, labels_list

all_preds, all_labels = predict_all(model, test_loader, DEVICE)


test_acc = (all_preds == all_labels).mean()
print(f"\n Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")


Loaded best model: best_resnet_ham10000.pt
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

 Test Accuracy: 0.7798 (77.98%)
